# Job Scheduling Simulator



<div> <h3>Team Members </h3>

CB.SC.U4AIE24024-K.usman

CB.SC.U4AIE24029-M.Praneeth babu

CB.SC.U4AIE24030-M.Jay krishna

CB.SC.U4AIE24032-M.Phanendra
</div>

 <summary>⏱ Time Complexity of Scheduler Methods

| **Function**                         | **Time Complexity** | **Description**                                                                 |
|--------------------------------------|----------------------|---------------------------------------------------------------------------------|
| `Task.__init__`                      | O(1)                 | Initializes a task with ID, name, burst time, arrival time, and priority.       |
| `Scheduler.__init__`                 | O(n)                 | Initializes the scheduler by copying tasks into an array.                      |
| `Scheduler.run`                      | O(1)                 | Abstract method that raises `NotImplementedError`.                             |
| `FIFOScheduler.selection_sort_by_arrival` | O(n²)           | Sorts tasks by arrival time using selection sort.                              |
| `FIFOScheduler.run`                  | O(n²)                | Sorts and executes tasks in FIFO order.                                        |
| `Node.__init__`                      | O(1)                 | Initializes a node with a task for the priority queue.                         |
| `PriorityQueue.__init__`            | O(1)                 | Initializes an empty priority queue.                                           |
| `PriorityQueue.insert_task`         | O(n)                 | Inserts task into the linked-list-based priority queue based on priority.      |
| `PriorityQueue.remove_min`          | O(1)                 | Removes and returns the task with the highest priority.                        |
| `PriorityQueue.is_empty`            | O(1)                 | Checks if the priority queue is empty.                                         |
| `PriorityScheduler.__init__`       | O(n)                 | Initializes the scheduler and priority queue.                                  |
| `PriorityScheduler.run`            | O(n²)                | Inserts tasks into priority queue and executes by priority.                    |
| `Array.__init__`                    | O(n)                 | Initializes an array for the circular queue.                                   |
| `Array.isFull`                      | O(1)                 | Checks if the circular queue is full.                                          |
| `Array.isEmpty`                     | O(1)                 | Checks if the circular queue is empty.                                         |
| `Array.insert`                      | O(1)                 | Inserts task into the circular queue.                                          |
| `Array.remove`                      | O(1)                 | Removes task from the circular queue.                                          |
| `CircularQueue.__init__`           | O(n)                 | Initializes a circular queue with an array.                                    |
| `CircularQueue.enqueue`            | O(1)                 | Adds a task to the circular queue.                                             |
| `CircularQueue.dequeue`            | O(1)                 | Removes and returns a task from the circular queue.                            |
| `CircularQueue.isEmpty`            | O(1)                 | Checks if the circular queue is empty.                                         |
| `RoundRobinScheduler.__init__`     | O(n)                 | Initializes tasks, quantum, and circular queue.                                |
| `RoundRobinScheduler.run`          | O(n + T/q)           | Executes tasks in round-robin style, time slicing with quantum.                |

 Where
 - **n** → Total number of tasks.  
- **T** → Total burst time of all tasks combined.  
- **q** →. Time quantum (the fixed amount of time each task is allowed to run per cycle).
</summary>

In [17]:
# ======================== TASK CLASS ========================
class Task:
    def __init__(self, ID, name, burst_time, arrival_time, priority=10):
        self.ID = ID
        self.name = name
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.arrival_time = arrival_time
        self.priority = priority
        self.start_time = -1
        self.completion_time = -1
        self.waiting_time = 0
        self.turnaround_time = 0


# ======================== BASE SCHEDULER CLASS ========================
class Scheduler:
    def __init__(self, tasks):
        n = 0
        for _ in tasks:
            n += 1
        self.tasks = [None] * n
        index = 0
        for task in tasks:
            self.tasks[index] = task
            index += 1

    def run(self):
        raise NotImplementedError("Subclasses must implement the run() method.")
        
# ======================== FIFO SCHEDULER ========================
def selection_sort_by_arrival(task_list):
    n = 0
    for _ in task_list:
        n += 1
    for i in range(n):
        min_index = i
        for j in range(i + 1, n):
            if task_list[j].arrival_time < task_list[min_index].arrival_time:
                min_index = j
        task_list[i], task_list[min_index] = task_list[min_index], task_list[i]

class FIFOScheduler(Scheduler):
    def run(self):
        selection_sort_by_arrival(self.tasks)
        current_time = 0
        for task in self.tasks:
            if current_time < task.arrival_time:
                current_time = task.arrival_time
            task.start_time = current_time
            print(f"FIFO: Executing {task.name} at time {current_time}")
            current_time += task.burst_time
            task.completion_time = current_time
            task.turnaround_time = task.completion_time - task.arrival_time
            task.waiting_time = task.start_time - task.arrival_time
            print(f"FIFO: {task.name} completed at time {current_time}")

        print("\nFIFO: All tasks completed!\n")
        for task in self.tasks:
            print(f"{task.name}: Waiting Time = {task.waiting_time}, Turnaround Time = {task.turnaround_time}")

# ======================== PRIORITY SCHEDULER ========================
class Node:
    def __init__(self, task):
        self.task = task
        self.next = None

class PriorityQueue:
    def __init__(self):
        self.head = None

    def insert_task(self, task):
        new_node = Node(task)
        if self.head is None:
            self.head = new_node
        elif (task.priority < self.head.task.priority) or (task.priority == self.head.task.priority and task.arrival_time < self.head.task.arrival_time):
            new_node.next = self.head
            self.head = new_node
        else:
            current = self.head
            while (current.next is not None and
                   (current.next.task.priority < task.priority or
                   (current.next.task.priority == task.priority and current.next.task.arrival_time <= task.arrival_time))):
                current = current.next
            new_node.next = current.next
            current.next = new_node

    def remove_min(self):
        if self.head is None:
            return None
        min_task = self.head.task
        self.head = self.head.next
        return min_task

    def is_empty(self):
        return self.head is None

class PriorityScheduler(Scheduler):
    def __init__(self, tasks):
        super().__init__(tasks)
        self.pq = PriorityQueue()

    def run(self):
        for task in self.tasks:
            self.pq.insert_task(task)

        current_time = 0
        completed_tasks = 0
        total_tasks = len(self.tasks)

        while completed_tasks < total_tasks:
            task = self.pq.remove_min()
            if task.start_time == -1:
                if current_time < task.arrival_time:
                    current_time = task.arrival_time
                task.start_time = current_time

            print(f"Priority: Executing {task.name} (Priority: {task.priority})")
            current_time += task.burst_time
            task.completion_time = current_time
            task.turnaround_time = task.completion_time - task.arrival_time
            task.waiting_time = task.start_time - task.arrival_time
            print(f"Priority: {task.name} completed at time {current_time}")
            completed_tasks += 1

        print("\nPriority: All tasks completed!\n")
        for task in self.tasks:
            print(f"{task.name}: Waiting Time = {task.waiting_time}, Turnaround Time = {task.turnaround_time}")

# ======================== ROUND ROBIN WITH CIRCULAR QUEUE ========================
class Array:
    def __init__(self, capacity):
        self.capacity = capacity
        self.array = [None] * capacity
        self.front = -1
        self.rear = -1

    def isFull(self):
        return (self.rear + 1) % self.capacity == self.front

    def isEmpty(self):
        return self.front == -1

    def insert(self, value):
        if self.isFull():
            print("Queue is full! Cannot insert.")
            return
        if self.isEmpty():
            self.front = 0
        self.rear = (self.rear + 1) % self.capacity
        self.array[self.rear] = value

    def remove(self):
        if self.isEmpty():
            return None
        value = self.array[self.front]
        if self.front == self.rear:
            self.front = -1
            self.rear = -1
        else:
            self.front = (self.front + 1) % self.capacity
        return value

class CircularQueue:
    def __init__(self, capacity):
        self.array = Array(capacity)

    def enqueue(self, value):
        self.array.insert(value)

    def dequeue(self):
        return self.array.remove()

    def isEmpty(self):
        return self.array.isEmpty()

class RoundRobinScheduler(Scheduler):
    def __init__(self, tasks, quantum):
        super().__init__(tasks)  
        self.quantum = quantum  
        self.queue = CircularQueue(len(tasks)) 

    def run(self):
        current_time = 0  # Track the current time
        remaining = len(self.tasks)  # Track the number of tasks left to execute

        # Enqueue all tasks to the queue
        for task in self.tasks:
            self.queue.enqueue(task)

        print("Round Robin: Starting execution...\n")


        while not self.queue.isEmpty():
            task = self.queue.dequeue()  

        
            if task.start_time == -1:
    
                if current_time < task.arrival_time:
                    current_time = task.arrival_time
                task.start_time = current_time

            # Manually select the execution time (quantum or remaining time
            if task.remaining_time <= self.quantum:
                exec_time = task.remaining_time  # Use remaining time if it's less than quantum
            else:
                exec_time = self.quantum  # Otherwise, use the quantum time

            print(f"RR: Executing {task.name} for {exec_time} units at time {current_time}")
            current_time += exec_time  # Update time after execution
            task.remaining_time -= exec_time  # Decrease remaining time

            # If the task is finished, calculate completion, turnaround, and waiting times
            if task.remaining_time == 0:
                task.completion_time = current_time
                task.turnaround_time = task.completion_time - task.arrival_time
                task.waiting_time = task.turnaround_time - task.burst_time
                print(f"RR: {task.name} completed at time {current_time}")
            else:
                # If the task isn't finished, put it back in the queue to execute later
                self.queue.enqueue(task)

        # After all tasks are processed, print the final statistics
        print("\nRound Robin: All tasks completed!\n")
        for task in self.tasks:
            print(f"{task.name}: Waiting Time = {task.waiting_time}, Turnaround Time = {task.turnaround_time}")

if __name__ == "__main__":
    tasks = [
        Task(1, "Task A", 10, 0, priority=2),
        Task(2, "Task B", 5, 2, priority=1),
        Task(3, "Task C", 8, 4, priority=3),
        Task(4, "Task D", 6, 1, priority=1)
    ]

    print("=== FIFO Scheduler ===")
    fifo_scheduler = FIFOScheduler(tasks)
    fifo_scheduler.run()

    print("\n=== Priority Scheduler ===")
    priority_scheduler = PriorityScheduler(tasks)
    priority_scheduler.run()

    print("\n=== Round Robin Scheduler (Quantum = 4) ===")
    round_robin_scheduler = RoundRobinScheduler(tasks,quantum=4)
    round_robin_scheduler.run()



=== FIFO Scheduler ===
FIFO: Executing Task A at time 0
FIFO: Task A completed at time 10
FIFO: Executing Task D at time 10
FIFO: Task D completed at time 16
FIFO: Executing Task B at time 16
FIFO: Task B completed at time 21
FIFO: Executing Task C at time 21
FIFO: Task C completed at time 29

FIFO: All tasks completed!

Task A: Waiting Time = 0, Turnaround Time = 10
Task D: Waiting Time = 9, Turnaround Time = 15
Task B: Waiting Time = 14, Turnaround Time = 19
Task C: Waiting Time = 17, Turnaround Time = 25

=== Priority Scheduler ===
Priority: Executing Task D (Priority: 1)
Priority: Task D completed at time 6
Priority: Executing Task B (Priority: 1)
Priority: Task B completed at time 11
Priority: Executing Task A (Priority: 2)
Priority: Task A completed at time 21
Priority: Executing Task C (Priority: 3)
Priority: Task C completed at time 29

Priority: All tasks completed!

Task A: Waiting Time = 0, Turnaround Time = 21
Task B: Waiting Time = 14, Turnaround Time = 9
Task C: Waiting T